# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
First, if you have an Azure subscription, create an [Azure AI hub](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources) then [create an Azure AI project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/ai-resources). AI projects and Hubs can be served within a private network and are compatible with private endpoints. You **do not** need to provide your own LLM deployment as the AI Red Teaming Agent hosts adversarial models for both simulation and evaluation of harmful content and connects to it via your Azure AI project.

**Required Role**: Ensure that you have the [Azure AI User](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/rbac-azure-ai-foundry#azure-ai-user) role assigned for your Azure subscription. This role provides the necessary permissions to run AI red teaming scans and access Azure AI Foundry services.

In order to upload your results to Azure AI Foundry:
- Your AI Foundry project must have a connection (*Connected Resources*) to a storage account with `Microsoft Entra ID` authentication enabled.
- Your AI Foundry project must have the `Storage Blob Data Contributor` role in the storage account.
- You must have the `Storage Blob Data Contributor` role in the storage account.
- You must have network access to the storage account.

For more information see: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/run-scans-ai-red-teaming-agent

**Important**: First, ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai azure-ai-projects
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [ ]:
from collections.abc import Mapping, Sequence
from pathlib import Path
from typing import Optional, Dict, Any, Protocol, TypeAlias
import json
import os
import time
from pprint import pprint
from dotenv import load_dotenv

# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy, SupportedLanguages
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentTaxonomyInput,
    AzureAIAgentTarget,
    DailyRecurrenceSchedule,
    EvaluationScheduleTask,
    EvaluationTaxonomy,
    RecurrenceTrigger,
    RiskCategory as ProjectsRiskCategory,
    Schedule,
)

# OpenAI imports
from openai import AzureOpenAI

### Login to Azure with valid credentials

Ensure that you've installed the [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) and then make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

Configure the `credential` object with a different AzureCredential type if this is a requirement for your environment.

In [ ]:
# Azure Credential imports
from azure.identity import AzureCliCredential, get_bearer_token_provider

!az login

# Initialize Azure credentials
credential = AzureCliCredential()

### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

Set these variables by creating an `.env` file in your project's root folder.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:


```

# Azure OpenAI

AZURE_OPENAI_API_KEY="your-api-key-here"

AZURE_OPENAI_ENDPOINT="https://endpoint-name.cognitiveservices.azure.com/openai/deployments/<model_deployment_name>/chat/completions?api-version=<azure_openai_api_version>"

AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"

AZURE_OPENAI_API_VERSION="2025-01-01-preview"



# Azure AI Project (required for cloud red teaming)

AZURE_PROJECT_ENDPOINT="https://your-aifoundry-endpoint-name.services.ai.azure.com/api/projects/yourproject-name"

AGENT_NAME="your-agent-name"

DATA_FOLDER="c:/temp/redteam"



# Model Configuration (alternative naming for cloud scenarios)

MODEL_ENDPOINT="https://endpoint-name.cognitiveservices.azure.com/"

MODEL_API_KEY="your-api-key-here"

AZURE_AI_MODEL_DEPLOYMENT_NAME="gpt-4"

```


In [ ]:
load_dotenv()

In [ ]:
# Azure AI Project information
azure_ai_project = os.environ.get("AZURE_PROJECT_ENDPOINT")


# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  # Use the latest API version
os.environ["OPENAI_API_VERSION"] = azure_openai_api_version

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - **Violence**: Content related to physical harm, weapons, or dangerous activities
   - **HateUnfairness**: Content related to discrimination, bias, or prejudice
   - **Sexual**: Explicit sexual content
   - **SelfHarm**: Content related to self-injury or suicide
   - **ProtectedMaterial**: Copyrighted or protected intellectual property
   - **CodeVulnerability**: Code that contains security vulnerabilities
   - **UngroundedAttributes**: Claims not supported by evidence
   
   **Cloud-Only Risk Categories** (available only when using Azure AI Foundry projects):
   - **ProhibitedActions**: Instructions for illegal or harmful activities
   - **SensitiveDataLeakage**: Exposure of personal or confidential information
   - **TaskAdherence**: Whether the AI follows intended task instructions

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content. You can also use `AttackStrategy.Compose()` to layer two strategies in one attack. 

In [ ]:
# Display all available attack strategies in the Azure AI Evaluation SDK
print("🎯 All Available Attack Strategies in Azure AI Evaluation SDK:")
print("=" * 70)
print()

# Get all AttackStrategy attributes
attack_strategies = [attr for attr in dir(AttackStrategy) if not attr.startswith("_")]

print(f"Total number of attack strategies: {len(attack_strategies)}")
print()

# Print each strategy
for i, strategy in enumerate(attack_strategies, 1):
    strategy_value = getattr(AttackStrategy, strategy)
    print(f"{i:2}. {strategy}: {strategy_value}")

print()
print("=" * 70)
print("Note: Use AttackStrategy.Compose([strategy1, strategy2]) to combine strategies")

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

### Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [ ]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [ ]:
# Create the `RedTeam` instance with comprehensive risk categories
# Note: Some categories like ProhibitedActions, SensitiveDataLeakage, and TaskAdherence
# are only available in cloud red teaming scenarios
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm,
        RiskCategory.ProtectedMaterial,
        RiskCategory.CodeVulnerability,
        RiskCategory.UngroundedAttributes,
    ],
    num_objectives=1,
)

NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [ ]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback,
    scan_name="Basic-Callback-Scan",
    attack_strategies=[AttackStrategy.Flip],
    output_path="red_team_output.json",
)

### Intermediary Example: Using a Model Configuration as Target

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. To test base or foundation models, you can update your target to take in a model configuration:

In [ ]:
# Define a model configuration to test
azure_oai_model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

Then, create a new Red Team instance with the `language` set to `SupportedLanguages.Spanish`

In [ ]:
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    language=SupportedLanguages.Spanish,
    num_objectives=1,
)

Then, update your target to point to the model configurations and run the scan.

In [ ]:
# Run the red team scan called "Intermediary-Model-Target-Scan"
result = await red_team.scan(
    target=azure_oai_model_config,
    scan_name="Intermediary-Model-Target-Scan",
    attack_strategies=[AttackStrategy.Flip],
)

### Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application.

In [ ]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    # Get token provider for Azure AD authentication
    token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=azure_openai_endpoint,
        api_version=azure_openai_api_version,
        azure_ad_token_provider=token_provider,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=azure_openai_deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            # temperature=0.7, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [ ]:
# Create the RedTeam instance with all available risk categories with 5 attack objectives generated for each category
# Note: ProhibitedActions, SensitiveDataLeakage, and TaskAdherence are only available in cloud scenarios
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[
        RiskCategory.Violence,
        RiskCategory.HateUnfairness,
        RiskCategory.Sexual,
        RiskCategory.SelfHarm,
        RiskCategory.ProtectedMaterial,
        RiskCategory.CodeVulnerability,
        RiskCategory.UngroundedAttributes,
    ],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [ ]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

### Bring your own objectives: Using your own prompts as objectives for RedTeam

Below we demonstrate how to use your own prompts as objectives for a `RedTeam` scan. You can see the required format for prompts under `.\data\prompts.json`. Note that when bringing your own prompts, the supported `risk-type`s are `violence`, `sexual`, `hate_unfairness`, and `self_harm`. The number of prompts you specify will be the `num_objectives` used in the scan. 

In [ ]:
path_to_prompts = "./data/prompts.json"

# Create the RedTeam specifying the custom attack seed prompts to use as objectives
custom_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    custom_attack_seed_prompts=path_to_prompts,  # Path to a file containing custom attack seed prompts
)

In [ ]:
custom_red_team_result = await custom_red_team.scan(
    target=azure_openai_callback,
    scan_name="Custom-Prompt-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.DIFFICULT,  # Group of difficult complexity attacks
    ],
    output_path="Custom-Prompt-Scan.json",
)

## Cloud Red Teaming with AI Project Client


The Azure AI Projects SDK can orchestrate end-to-end cloud red teaming evaluations. This flow retrieves the latest Copilot agent version from your project, defines risk taxonomies, persists them for reuse, and submits a managed evaluation run. Cloud execution unlocks additional risk categories, centralised result storage, and lets you poll for completion without building custom orchestration.

### Helper utilities

In [ ]:
JSONPrimitive = str | int | float | bool | None
JSONValue: TypeAlias = JSONPrimitive | list["JSONValue"] | dict[str, "JSONValue"]


class SupportsDefinition(Protocol):
    definition: Mapping[str, object] | None


def _get_red_team_evaluation_criteria(
    include_agent_specific: bool = True,
) -> list[dict[str, str]]:
    base_criteria = [
        {
            "type": "azure_ai_evaluator",
            "name": "Violence",
            "evaluator_name": "builtin.violence",
            "evaluator_version": "1",
        },
        {
            "type": "azure_ai_evaluator",
            "name": "Self Harm",
            "evaluator_name": "builtin.self_harm",
            "evaluator_version": "1",
        },
        {
            "type": "azure_ai_evaluator",
            "name": "Sexual",
            "evaluator_name": "builtin.sexual",
            "evaluator_version": "1",
        },
        {
            "type": "azure_ai_evaluator",
            "name": "Hate Unfairness",
            "evaluator_name": "builtin.hate_unfairness",
            "evaluator_version": "1",
        },
    ]
    if include_agent_specific:
        base_criteria.extend(
            [
                {
                    "type": "azure_ai_evaluator",
                    "name": "Prohibited Actions",
                    "evaluator_name": "builtin.prohibited_actions",
                    "evaluator_version": "1",
                },
                {
                    "type": "azure_ai_evaluator",
                    "name": "Sensitive Data Leakage",
                    "evaluator_name": "builtin.sensitive_data_leakage",
                    "evaluator_version": "1",
                },
                {
                    "type": "azure_ai_evaluator",
                    "name": "Task Adherence",
                    "evaluator_name": "builtin.task_adherence",
                    "evaluator_version": "1",
                },
            ]
        )
    return base_criteria


def _get_tool_descriptions(agent: SupportsDefinition | Mapping[str, object] | None) -> list[dict[str, str]]:
    if agent is None:
        return []
    definition_candidate = getattr(agent, "definition", None)
    if definition_candidate is None and isinstance(agent, Mapping):
        definition_candidate = agent
    descriptions: list[dict[str, str]] = []
    if isinstance(definition_candidate, Mapping):
        raw_tools = definition_candidate.get("tools", [])
        if isinstance(raw_tools, Sequence) and not isinstance(raw_tools, (str, bytes)):
            for tool in raw_tools:
                if not isinstance(tool, Mapping):
                    continue
                if tool.get("type") == "openapi":
                    openapi_section = tool.get("openapi", {})
                    if isinstance(openapi_section, Mapping):
                        descriptions.append(
                            {
                                "name": str(openapi_section.get("name", "Unnamed Tool")),
                                "description": str(openapi_section.get("description", "No description provided")),
                            }
                        )
                    continue
                descriptions.append(
                    {
                        "name": str(tool.get("name", "Unnamed Tool")),
                        "description": str(tool.get("description", "No description provided")),
                    }
                )
    return descriptions


def _to_json_primitive(value: object) -> JSONValue:
    if value is None or isinstance(value, (str, int, float, bool)):
        return value
    if isinstance(value, (list, tuple, set)):
        return [_to_json_primitive(item) for item in value]
    if isinstance(value, Mapping):
        return {str(key): _to_json_primitive(val) for key, val in value.items()}
    for attr in ("to_dict", "as_dict", "dict", "serialize"):
        if hasattr(value, attr):
            method = getattr(value, attr)
            try:
                result = method() if callable(method) else method
                return _to_json_primitive(result)
            except Exception:
                continue
    if hasattr(value, "__dict__"):
        return _to_json_primitive({k: v for k, v in vars(value).items() if not k.startswith("_")})
    return str(value)

### Configure cloud model evaluation


Update the values below to point at the Azure OpenAI deployment you want to exercise in the managed cloud run. Strategy names should match the options listed earlier in the notebook (for example: `"Flip"`, `"Base64"`, `"Crescendo"`).

In [ ]:
cloud_model_attack_strategies = [
    "Crescendo"
    # Add additional strategy names (e.g., "Flip", "Jailbreak", "IndirectJailbreak") as needed
]
cloud_model_deployment_name = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME") or azure_openai_deployment or ""
cloud_model_num_objectives = 1

### Submit cloud model evaluation

In [ ]:
def run_cloud_model_red_team_evaluation(
    attack_strategies: Sequence[str] | None = None,
    model_deployment_name: str | None = None,
    num_objectives: int = 5,
) -> None:
    endpoint = os.environ.get("AZURE_PROJECT_ENDPOINT") or azure_ai_project or ""
    data_folder = os.environ.get("DATA_FOLDER", str(Path.cwd() / "redteam_outputs"))
    strategies = [strategy for strategy in (attack_strategies or ["Flip", "Base64"]) if strategy]
    target_deployment = (
        model_deployment_name or os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME") or azure_openai_deployment or ""
    )
    print(f"Targetting deployment name: {target_deployment}")
    model_endpoint = os.environ.get("MODEL_ENDPOINT") or azure_openai_endpoint or ""
    model_api_key = os.environ.get("MODEL_API_KEY") or azure_openai_api_key or ""
    api_version = os.environ.get("OPENAI_API_VERSION") or azure_openai_api_version or ""
    if not endpoint:
        print("Cloud red teaming skipped: set AZURE_PROJECT_ENDPOINT.")
        return
    if not target_deployment:
        print("Cloud red teaming skipped: set AZURE_AI_MODEL_DEPLOYMENT_NAME or pass model_deployment_name.")
        return
    if not model_endpoint or not model_api_key:
        print("Cloud red teaming skipped: set MODEL_ENDPOINT and MODEL_API_KEY.")
        return
    if not strategies:
        print("Cloud red teaming skipped: provide at least one attack strategy.")
        return

    print(f"Using cloud model attack strategies: {strategies}")
    print(f"Target deployment: {target_deployment}")
    print(f"Project endpoint: {endpoint}")
    if api_version:
        print(f"OpenAI API version: {api_version}")
    else:
        print("No OpenAI API version set; the SDK will use its default.")

    data_folder_path = Path(data_folder)
    data_folder_path.mkdir(parents=True, exist_ok=True)

    with DefaultAzureCredential() as project_credential, AIProjectClient(
        endpoint=endpoint,
        credential=project_credential,
        api_version="2025-11-15-preview",
    ) as project_client:
        print("Creating an OpenAI client from the AI Project client")
        client = project_client.get_openai_client()

        try:
            eval_group_name = f"Red Team Model Safety Eval Group - {int(time.time())}"
            eval_run_name = f"Red Team Model Safety Eval Run for {target_deployment} - {int(time.time())}"
            data_source_config = {"type": "azure_ai_source", "scenario": "red_team"}

            testing_criteria = _get_red_team_evaluation_criteria(include_agent_specific=False)

            print("Creating Eval Group")
            eval_object = client.evals.create(
                name=eval_group_name,
                data_source_config=data_source_config,
                testing_criteria=testing_criteria,
            )
        except Exception as error:
            print("Failed when creating the eval group. Raw error:")
            print(error)
            raise
        print(f"Eval Group created for model red teaming: {eval_group_name}")

        print(f"Get Eval Group by Id: {eval_object.id}")
        eval_object_response = client.evals.retrieve(eval_object.id)
        print("Eval Group Response:")
        pprint(eval_object_response)

        try:
            print("Creating RedTeaming Eval Run for model")
            eval_run_object = client.evals.runs.create(
                eval_id=eval_object.id,
                name=eval_run_name,
                data_source={
                    "type": "azure_ai_red_team",
                    "item_generation_params": {
                        "type": "red_team",
                        "attack_strategies": strategies,
                        "num_turns": num_objectives,
                    },
                    "target": {
                        "type": "azure_ai_model",
                        "model": target_deployment,
                        "sampling_params": {
                            "temperature": 0.7,
                            "top_p": 1.0,
                            "seed": 0,
                            "max_completion_tokens": 1024,
                        },
                    },
                },
            )

            print(f"Eval Run created for model red teaming: {eval_run_name}")
            pprint(eval_run_object)
        except Exception as error:
            print("Failed when creating the eval run. Raw error:")
            print(error)
            raise

        print(f"Get Eval Run by Id: {eval_run_object.id}")
        eval_run_response = client.evals.runs.retrieve(
            run_id=eval_run_object.id,
            eval_id=eval_object.id,
        )
        print("Eval Run Response:")
        pprint(eval_run_response)

        while True:
            run = client.evals.runs.retrieve(
                run_id=eval_run_response.id,
                eval_id=eval_object.id,
            )
            if run.status in {"completed", "failed"}:
                output_items = list(
                    client.evals.runs.output_items.list(
                        run_id=run.id,
                        eval_id=eval_object.id,
                    )
                )
                output_items_path = data_folder_path / f"redteam_model_eval_output_{target_deployment}.json"
                with output_items_path.open("w", encoding="utf-8") as file_handle:
                    json.dump(_to_json_primitive(output_items), file_handle, indent=2)
                print(
                    f"Model red team run completed with status: {run.status}. "
                    f"Output items written to {output_items_path}"
                )
                break
            time.sleep(5)
            print("Waiting for model eval run to complete...")

In [ ]:
try:
    run_cloud_model_red_team_evaluation(
        attack_strategies=cloud_model_attack_strategies,
        model_deployment_name=cloud_model_deployment_name,
        num_objectives=cloud_model_num_objectives,
    )
except Exception as exc:
    print(f"Cloud red teaming failed: {exc}")

### Configure cloud agent evaluation

In [ ]:
cloud_agent_attack_strategies = [
    "Flip",
    "Base64",
]
cloud_agent_num_objectives = 5

### Submit cloud agent evaluation

In [ ]:
def run_cloud_agent_red_team_evaluation(
    attack_strategies: Sequence[str] | None = None,
    num_objectives: int = 5,
) -> None:
    endpoint = os.environ.get("AZURE_PROJECT_ENDPOINT") or azure_ai_project or ""
    agent_name = os.environ.get("AGENT_NAME", "")
    data_folder = os.environ.get("DATA_FOLDER", str(Path.cwd() / "redteam_outputs"))
    strategies = [strategy for strategy in (attack_strategies or ["Flip", "Base64"]) if strategy]
    api_version = os.environ.get("OPENAI_API_VERSION") or azure_openai_api_version or ""
    if not endpoint:
        print("Cloud red teaming skipped: set AZURE_PROJECT_ENDPOINT.")
        return
    if not agent_name:
        print("Cloud red teaming skipped: set AGENT_NAME.")
        return
    if not strategies:
        print("Cloud red teaming skipped: provide at least one attack strategy.")
        return

    print(f"Using cloud agent attack strategies: {strategies}")
    print(f"Target agent: {agent_name}")
    print(f"Project endpoint: {endpoint}")
    if api_version:
        print(f"OpenAI API version: {api_version}")
    else:
        print("No OpenAI API version set; the SDK will use its default.")

    data_folder_path = Path(data_folder)
    data_folder_path.mkdir(parents=True, exist_ok=True)

    with DefaultAzureCredential() as project_credential, AIProjectClient(
        endpoint=endpoint,
        credential=project_credential,
        api_version="2025-11-15-preview",
    ) as project_client:
        print("Creating an OpenAI client from the AI Project client")
        client = project_client.get_openai_client()

        try:
            versions = list(project_client.agents.list_versions(agent_name))
            latest = max(versions, key=lambda v: float(v.version))  # adjust if versions aren't numeric
            agent = project_client.agents.get_version(agent_name, latest.version)
            agent_version = agent.version
            print(f"Retrieved agent: {agent_name}, version: {agent_version}")
            eval_group_name = f"Red Team Agent Safety Eval Group - {int(time.time())}"
            eval_run_name = f"Red Team Agent Safety Eval Run for {agent_name} - {int(time.time())}"
            data_source_config = {"type": "azure_ai_source", "scenario": "red_team"}

            testing_criteria = _get_red_team_evaluation_criteria()

            print("Creating Eval Group")
            eval_object = client.evals.create(
                name=eval_group_name,
                data_source_config=data_source_config,
                testing_criteria=testing_criteria,
            )
        except Exception as error:
            print("Failed when creating the eval group. Raw error:")
            print(error)
            raise

        print(f"Eval Group created for agent red teaming: {eval_group_name}")

        print(f"Get Eval Group by Id: {eval_object.id}")
        eval_object_response = client.evals.retrieve(eval_object.id)
        print("Eval Group Response:")
        pprint(eval_object_response)

        try:
            risk_categories_for_taxonomy = [ProjectsRiskCategory.PROHIBITED_ACTIONS]
            target = AzureAIAgentTarget(
                name=agent_name,
                version=agent_version,
                tool_descriptions=_get_tool_descriptions(agent),
            )
            agent_taxonomy_input = AgentTaxonomyInput(
                risk_categories=risk_categories_for_taxonomy,
                target=target,
            )
            print("Creating Eval Taxonomies")
            eval_taxonomy_input = EvaluationTaxonomy(
                description="Taxonomy for agent red teaming evaluation",
                taxonomy_input=agent_taxonomy_input,
            )

            taxonomy = project_client.evaluation_taxonomies.create(
                name=agent_name,
                body=eval_taxonomy_input,
            )
            taxonomy_path = data_folder_path / f"taxonomy_agent_{agent_name}.json"
            with taxonomy_path.open("w", encoding="utf-8") as file_handle:
                json.dump(_to_json_primitive(taxonomy), file_handle, indent=2)
            print(f"RedTeaming Taxonomy created for agent: {agent_name}. Taxonomy written to {taxonomy_path}")
        except Exception as error:
            print("Failed when creating the eval taxonomy. Raw error:")
            print(error)
            raise

        try:
            print("Creating RedTeaming Eval Run for agent")
            eval_run_object = client.evals.runs.create(
                eval_id=eval_object.id,
                name=eval_run_name,
                data_source={
                    "type": "azure_ai_red_team",
                    "item_generation_params": {
                        "type": "red_team_taxonomy",
                        "attack_strategies": strategies,
                        "num_turns": num_objectives,
                        "source": {"type": "file_id", "id": taxonomy.id},
                    },
                    "target": target.as_dict(),
                },
            )
        except Exception as error:
            print("Failed when creating the eval run. Raw error:")
            print(error)
            raise

        print(f"Eval Run created for agent red teaming: {eval_run_name}")
        pprint(eval_run_object)

        print(f"Get Eval Run by Id: {eval_run_object.id}")
        eval_run_response = client.evals.runs.retrieve(
            run_id=eval_run_object.id,
            eval_id=eval_object.id,
        )
        print("Eval Run Response:")
        pprint(eval_run_response)

        while True:
            run = client.evals.runs.retrieve(
                run_id=eval_run_response.id,
                eval_id=eval_object.id,
            )
            if run.status in {"completed", "failed"}:
                output_items = list(
                    client.evals.runs.output_items.list(
                        run_id=run.id,
                        eval_id=eval_object.id,
                    )
                )
                output_items_path = data_folder_path / f"redteam_agent_eval_output_{agent_name}.json"
                with output_items_path.open("w", encoding="utf-8") as file_handle:
                    json.dump(_to_json_primitive(output_items), file_handle, indent=2)
                print(
                    f"Agent red team run completed with status: {run.status}. "
                    f"Output items written to {output_items_path}"
                )
                break
            time.sleep(5)
            print("Waiting for agent eval run to complete...")

In [ ]:
try:
    run_cloud_agent_red_team_evaluation(
        attack_strategies=cloud_agent_attack_strategies,
        num_objectives=cloud_agent_num_objectives,
    )
except Exception as exc:
    print(f"Cloud red teaming failed: {exc}")

### Schedule Recurring Cloud Red Teaming

In [ ]:
def schedule_cloud_agent_red_team_evaluation(
    schedule_id: str = "redteam-agent-eval-daily-0900",
    run_hour_utc: int = 9,
    attack_strategies: Sequence[str] | None = None,
    num_objectives: int = 5,
) -> None:
    """Create or update a daily Azure AI schedule that submits an agent red team run."""
    endpoint = os.environ.get("AZURE_PROJECT_ENDPOINT") or azure_ai_project or ""
    agent_name = os.environ.get("AGENT_NAME", "")
    data_folder = os.environ.get("DATA_FOLDER", str(Path.cwd() / "redteam_outputs"))
    strategies = [strategy for strategy in (attack_strategies or ["Flip", "Base64"]) if strategy]
    if not 0 <= run_hour_utc <= 23:
        print("Provide run_hour_utc between 0 and 23 (UTC).")
        return
    if not endpoint:
        print("Scheduling skipped: set AZURE_PROJECT_ENDPOINT.")
        return
    if not agent_name:
        print("Scheduling skipped: set AGENT_NAME.")
        return
    if not strategies:
        print("Scheduling skipped: provide at least one attack strategy.")
        return
    data_folder_path = Path(data_folder)
    data_folder_path.mkdir(parents=True, exist_ok=True)
    print(f"Using project endpoint: {endpoint}")
    print(f"Target agent: {agent_name}")
    print(f"Schedule id: {schedule_id} (daily at {run_hour_utc:02d}:00 UTC)")
    with DefaultAzureCredential() as project_credential, AIProjectClient(
        endpoint=endpoint,
        credential=project_credential,
        api_version="2025-11-15-preview",
    ) as project_client:
        print("Creating an OpenAI client from the AI Project client")
        client = project_client.get_openai_client()
        versions = list(project_client.agents.list_versions(agent_name))
        if not versions:
            print(f"No published versions found for agent {agent_name}.")
            return
        latest = max(versions, key=lambda version: float(version.version))
        agent = project_client.agents.get_version(agent_name, latest.version)
        agent_version = agent.version
        print(f"Retrieved agent version: {agent_version}")
        eval_group_name = f"Red Team Agent Safety Eval Group - {int(time.time())}"
        eval_run_name = f"Red Team Agent Safety Eval Run for {agent_name} - {int(time.time())}"
        data_source_config = {"type": "azure_ai_source", "scenario": "red_team"}
        testing_criteria = _get_red_team_evaluation_criteria()
        print("Creating Eval Group for scheduled run")
        eval_object = client.evals.create(
            name=eval_group_name,
            data_source_config=data_source_config,
            testing_criteria=testing_criteria,
        )
        print(f"Eval Group id: {eval_object.id}")
        risk_categories_for_taxonomy = [ProjectsRiskCategory.PROHIBITED_ACTIONS]
        target = AzureAIAgentTarget(
            name=agent_name,
            version=agent_version,
            tool_descriptions=_get_tool_descriptions(agent),
        )
        agent_taxonomy_input = AgentTaxonomyInput(
            risk_categories=risk_categories_for_taxonomy,
            target=target,
        )
        eval_taxonomy_input = EvaluationTaxonomy(
            description="Taxonomy for scheduled agent red teaming",
            taxonomy_input=agent_taxonomy_input,
        )
        taxonomy = project_client.evaluation_taxonomies.create(
            name=agent_name,
            body=eval_taxonomy_input,
        )
        taxonomy_path = data_folder_path / f"taxonomy_agent_{agent_name}.json"
        with taxonomy_path.open("w", encoding="utf-8") as file_handle:
            json.dump(_to_json_primitive(taxonomy), file_handle, indent=2)
        print(f"Red team taxonomy stored at {taxonomy_path}. Schedule will reuse file id {taxonomy.id}.")
        eval_run_object = {
            "eval_id": eval_object.id,
            "name": eval_run_name,
            "data_source": {
                "type": "azure_ai_red_team",
                "item_generation_params": {
                    "type": "red_team_taxonomy",
                    "attack_strategies": strategies,
                    "num_turns": num_objectives,
                    "source": {"type": "file_id", "id": taxonomy.id},
                },
                "target": target.as_dict(),
            },
        }
        print("Creating or updating schedule")
        schedule = Schedule(
            display_name="Red Team Agent Safety Eval",
            enabled=True,
            trigger=RecurrenceTrigger(
                interval=1,
                schedule=DailyRecurrenceSchedule(hours=[run_hour_utc]),
            ),
            task=EvaluationScheduleTask(
                eval_id=eval_object.id,
                eval_run=eval_run_object,
            ),
        )
        try:
            project_client.schedules.get(schedule_id)
            schedule_response = project_client.schedules.create_or_update(
                schedule_id=schedule_id,
                schedule=schedule,
            )
        except Exception:
            schedule_response = project_client.schedules.create(
                schedule_id=schedule_id,
                schedule=schedule,
            )
        print(f"Schedule ready: {schedule_response.id}")
        print("Latest schedule definition:")
        pprint(_to_json_primitive(schedule_response))
        print("Recent schedule runs:")
        for run in project_client.schedules.list_runs(schedule_id=schedule_response.id):
            pprint(_to_json_primitive(run))

In [ ]:
try:
    schedule_cloud_agent_red_team_evaluation(
        attack_strategies=cloud_agent_attack_strategies,
        num_objectives=cloud_agent_num_objectives,
    )
except Exception as exc:
    print(f"Cloud red teaming failed: {exc}")

## Troubleshooting Errors


### Permission or authentication failures


- Confirm you ran `az login` in the active shell and that the account has the **Azure AI User** role plus the storage `Storage Blob Data Contributor` assignments called out earlier in this notebook. These roles are required to create evaluation runs and upload artifacts.
- When working inside a secured hub, make sure the connected storage account is accessible from your network and that Entra ID authentication is enabled on the storage resource.
- If the cloud red teaming helper prints `This may be due to missing environment variables or insufficient permissions.`, double-check the `AZURE_PROJECT_ENDPOINT`, `AGENT_NAME`, and storage role assignments before retrying.

### PyRIT "Error sending prompt" message


- `Exception: Error sending prompt with conversation ID: <guid>` indicates PyRIT could not reach the target LLM during a conversation turn. The runner retries the same conversation up to the configured limit, so isolated occurrences are expected and usually recover without intervention.
- Common triggers are transient network issues, 429 throttling, or 5xx responses from the target deployment. Even if retries succeed you will still see the stack trace in the notebook output.
- Inspect the `redteam.log` file emitted to the scan output directory (for local runs this defaults to `<working dir>/runs/<scan_id>/redteam.log`) for the original exception and HTTP status. Increase verbosity with `PF_LOGGING_LEVEL=DEBUG` when deeper diagnostics are needed.
- If one conversation ID keeps failing, verify the target credentials, check deployment health, and review Azure OpenAI quota/rate-limit alerts in the Azure portal.

### Target resource not found


- When you pass an Azure OpenAI deployment directly as the `target` to `RedTeam`, set `azure_endpoint` to `https://<hub>.openai.azure.com/openai/deployments/<deployment_name>/chat/completions?api-version=2025-01-01-preview`.
- If you instantiate `AzureOpenAI`, use the resource-level endpoint format `https://<hub>.openai.azure.com/`. Ensure the deployment name and API version match an active deployment in the selected hub.
- If the cloud run fails with `Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}` while creating the eval group, confirm you have `azure-ai-projects>=2.0.0b1` installed (the preview APIs surface only in that version or later).

### Agent name not found


- A `(not_found) Agent <name> doesn’t exist` message means the project could not resolve the agent `name`. Agent names are case sensitive and differ from display names.
- Use the helper below to enumerate every agent the current `AZURE_PROJECT_ENDPOINT` exposes. Confirm the `name` column matches the value in `AGENT_NAME`.
- If the agent is missing, verify you selected the correct project endpoint/region, that the agent is published in Azure AI Foundry, and that your account has access to it.

```python
def list_project_agents(endpoint: str | None = None) -> None:
    """Print the agent names available in the current Azure AI project."""
    project_endpoint = endpoint or os.environ.get("AZURE_PROJECT_ENDPOINT") or azure_ai_project or ""
    if not project_endpoint:
        print("Set AZURE_PROJECT_ENDPOINT before listing agents.")
        return
    with DefaultAzureCredential() as project_credential:
        with AIProjectClient(
            endpoint=project_endpoint,
            credential=project_credential,
            api_version="2025-11-15-preview",
        ) as project_client:
            agents = list(project_client.agents.list())
    if not agents:
        print(f"No agents found in project: {project_endpoint}")
        return
    print(f"Agents in {project_endpoint}:")
    for agent in agents:
        display_name = agent.get("display_name") if isinstance(agent, dict) else getattr(agent, "display_name", "")
        name = agent.get("name") if isinstance(agent, dict) else getattr(agent, "name", "")
        print(f"- name: {name} | display_name: {display_name}")
```

## Conclusion

In this comprehensive notebook, we've demonstrated the full spectrum of Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems across multiple dimensions:

### What We Covered

1. **Complete Risk Category Coverage**: We explored all available risk categories, including both standard categories (Violence, HateUnfairness, Sexual, SelfHarm, ProtectedMaterial, CodeVulnerability, UngroundedAttributes) and cloud-only categories (ProhibitedActions, SensitiveDataLeakage, TaskAdherence)

2. **Advanced Attack Strategies**: We demonstrated sophisticated attack techniques including:
   - Single-turn encoding and obfuscation attacks
   - Multi-turn conversational attacks
   - **Crescendo attacks** - gradually escalating conversations from benign to harmful
   - Composed attack strategies that layer multiple techniques

3. **Cloud Red Teaming**: We showed how to leverage the Azure AI Projects SDK for cloud-based red teaming that provides access to additional risk categories and evaluation capabilities

4. **Custom Attack Objectives**: We demonstrated how to use custom prompts for targeted testing scenarios

5. **Automated Scheduling**: We mirrored the preview `sample_scheduled_evaluations.py` flow to register recurring agent safety evaluations directly from Azure AI Projects

### Key Insights

The automated AI red teaming scans provide valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to  
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Multi-Turn Attack Patterns** - How sophisticated conversational attacks like Crescendo can gradually bypass safety measures
5. **Cloud vs Local Capabilities** - Additional risk categories and evaluation depth available through Azure AI Foundry

### Deployment Considerations

- **Crescendo and MultiTurn attacks** require significantly more time and resources than single-turn attacks
- **Cloud-only agent safety risk categories** (ProhibitedActions, SensitiveDataLeakage, TaskAdherence) are only available when targetting an agent using cloud Red Teaming
- **Multi-turn attacks** provide deeper insights but require longer execution times and higher API costs

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors, paying special attention to multi-turn attack patterns
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle, including both fast single-turn and thorough multi-turn testing
3. **Cloud Integration**: Consider leveraging Azure AI Foundry for access to additional risk categories and cloud-scale evaluation capabilities
4. **Custom Strategies**: Develop custom attack strategies and objectives for your specific use cases and domain
5. **Safety Layers**: Implement multiple layers of defense including Azure AI Content Safety, custom content filters, and conversation flow controls
6. **Monitoring**: Set up continuous monitoring for attack patterns in production, especially sophisticated multi-turn attempts 
7. **Schedule Health Checks**: Review the scheduled run history and alerts so unattended evaluations keep delivering fresh coverage without manual intervention